# BlockAMC: Scalability Algorithm for Large Matrices

This notebook demonstrates the BlockAMC scalability approach for handling large matrices using the Block HP-INV algorithm.

In [ ]:
# Import required libraries
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

# Add the project root to the path
sys.path.insert(0, os.path.join(os.getcwd()))

from src.hp_inv import hp_inv, block_hp_inv
from src.rram_model import create_rram_matrix
from src.redundancy import apply_redundancy

# Set up plotting style
plt.style.use('seaborn-v0_8')
np.random.seed(42)

## 1. Introduction to BlockAMC Concept

BlockAMC (Block Analogue Matrix Computing) partitions large matrices into block submatrices, allowing 8x8 RRAM arrays to solve larger systems without reprogramming. This notebook demonstrates an implementation of this approach.

In [ ]:
# Create a larger matrix to demonstrate block partitioning
large_n = 16
block_size = 8

print(f"Creating a {large_n}x{large_n} matrix to demonstrate BlockAMC")
print(f"This will be partitioned into blocks of size {block_size}x{block_size}")

# Generate a large RRAM matrix with realistic parameters
G_large = create_rram_matrix(large_n, variability=0.03, stuck_fault_prob=0.015, line_resistance=1.7e-3)
# Add diagonal dominance to ensure the matrix is well-conditioned
G_large = G_large + 0.4 * np.eye(large_n)

# Apply redundancy to handle stuck-at faults
G_large_repaired = apply_redundancy(G_large)

print(f"Original matrix condition number: {np.linalg.cond(G_large):.2f}")
print(f"Repaired matrix condition number: {np.linalg.cond(G_large_repaired):.2f}")

# Visualize the large matrix
plt.figure(figsize=(8, 6))
plt.imshow(G_large_repaired, cmap='viridis', aspect='equal')
plt.colorbar(label='Conductance (S)')
plt.title(f'{large_n}x{large_n} RRAM Matrix (After Redundancy Application)')
plt.show()

## 2. Comparing Standard HP-INV vs Block HP-INV

Let's compare the performance of standard HP-INV with Block HP-INV on the large matrix.

In [ ]:
# Generate right-hand side vector
b_large = np.random.randn(large_n)

# For reference, solve using standard numpy (for well-conditioned system)
x_true_large = np.linalg.solve(G_large_repaired, b_large)

# Solve using standard HP-INV (this may be slower for larger matrices)
print("Solving with standard HP-INV...")
x_std, iters_std, info_std = hp_inv(G_large_repaired, b_large, bits=3, lp_noise_std=0.01, max_iter=25, tol=1e-5)
rel_error_std = np.linalg.norm(x_std - x_true_large) / np.linalg.norm(x_true_large)

# Solve using Block HP-INV
print("Solving with Block HP-INV...")
x_block, iters_block, info_block = block_hp_inv(
    G_large_repaired, 
    b_large, 
    block_size=block_size, 
    bits=3, 
    lp_noise_std=0.01, 
    max_iter=25, 
    tol=1e-5
)
rel_error_block = np.linalg.norm(x_block - x_true_large) / np.linalg.norm(x_true_large)

# Compare results
print(f"\nStandard HP-INV Results:")
print(f"  Iterations: {iters_std}")
print(f"  Relative Error: {rel_error_std:.2e}")
print(f"  Final Residual: {info_std['final_residual']:.2e}")
print(f"  Converged: {info_std['converged']}")

print(f"\nBlock HP-INV Results:")
print(f"  Iterations: {iters_block}")
print(f"  Relative Error: {rel_error_block:.2e}")
print(f"  Converged: {info_block['converged']}")

# Plot comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot convergence residuals
ax1.semilogy(info_std['residuals'], 'o-', label='Standard HP-INV', linewidth=2, markersize=6)
if 'residuals' in info_block:
    ax1.semilogy(info_block['residuals'], 's-', label='Block HP-INV', linewidth=2, markersize=6)
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Residual Norm')
ax1.set_title('Convergence Comparison')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot solutions comparison
ax2.plot(x_true_large, 'o-', label='True Solution', linewidth=2, markersize=4)
ax2.plot(x_std, 's-', label=f'Std HP-INV (err: {rel_error_std:.2e})', linewidth=2, markersize=4)
ax2.plot(x_block, '^-', label=f'Block HP-INV (err: {rel_error_block:.2e})', linewidth=2, markersize=4)
ax2.set_xlabel('Index')
ax2.set_ylabel('Value')
ax2.set_title('Solution Comparison')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Performance Analysis of BlockAMC with Different Block Sizes

Let's analyze how different block sizes affect the performance of the Block HP-INV algorithm.

In [ ]:
# Test with different block sizes
matrix_size = 16
G_test = create_rram_matrix(matrix_size, variability=0.02, stuck_fault_prob=0.01)
G_test = G_test + 0.3 * np.eye(matrix_size)  # Add diagonal dominance
G_test = apply_redundancy(G_test)
b_test = np.random.randn(matrix_size)

# True solution for comparison
x_true = np.linalg.solve(G_test, b_test)

# Define different block sizes to test
block_sizes = [4, 8, 16]  # 16 would mean no blocking

# Store results
results = []

for block_size in block_sizes:
    if block_size >= matrix_size:
        # For block size >= matrix size, use standard HP-INV
        x, iters, info = hp_inv(G_test, b_test, bits=3, lp_noise_std=0.01, max_iter=20, tol=1e-5)
    else:
        x, iters, info = block_hp_inv(
            G_test, b_test, block_size=block_size, 
            bits=3, lp_noise_std=0.01, max_iter=20, tol=1e-5
        )
    
    rel_error = np.linalg.norm(x - x_true) / np.linalg.norm(x_true)
    results.append({
        'block_size': block_size,
        'iterations': iters,
        'error': rel_error,
        'converged': info.get('converged', False)
    })
    
    print(f"Block size {block_size}: {iters} iterations, error: {rel_error:.2e}, converged: {info.get('converged', False)}")

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot iterations vs block size
ax1.plot([r['block_size'] for r in results], [r['iterations'] for r in results], 
         'o-', linewidth=2, markersize=8)
ax1.set_xlabel('Block Size')
ax1.set_ylabel('Iterations to Convergence')
ax1.set_title('Convergence Speed vs Block Size')
ax1.grid(True, alpha=0.3)

# Plot error vs block size
ax2.plot([r['block_size'] for r in results], [r['error'] for r in results], 
         's-', linewidth=2, markersize=8)
ax2.set_xlabel('Block Size')
ax2.set_ylabel('Relative Error')
ax2.set_yscale('log')
ax2.set_title('Solution Accuracy vs Block Size')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Scalability Analysis

Let's examine how the BlockAMC approach scales with matrix size.

In [ ]:
# Test scalability with different matrix sizes
matrix_sizes = [8, 12, 16, 20]
block_size = 8
n_tests = 5  # Number of tests per size to average results

# Store scalability results
scalability_results = []

for size in matrix_sizes:
    std_times = []
    block_times = []
    std_errors = []
    block_errors = []
    
    print(f"Testing matrix size {size}x{size}...")
    
    for i in range(n_tests):
        # Create test matrix
        G = create_rram_matrix(size, variability=0.02, stuck_fault_prob=0.01)
        G = G + 0.3 * np.eye(size)  # Add diagonal dominance
        G = apply_redundancy(G)
        b = np.random.randn(size)
        x_true = np.linalg.solve(G, b)
        
        # Time standard HP-INV
        import time
        start_time = time.time()
        x_std, iters_std, _ = hp_inv(G, b, bits=3, lp_noise_std=0.01, max_iter=15, tol=1e-4)
        std_time = time.time() - start_time
        std_error = np.linalg.norm(x_std - x_true) / np.linalg.norm(x_true)
        
        # Time Block HP-INV
        start_time = time.time()
        x_block, iters_block, _ = block_hp_inv(G, b, block_size=block_size, 
                                               bits=3, lp_noise_std=0.01, max_iter=15, tol=1e-4)
        block_time = time.time() - start_time
        block_error = np.linalg.norm(x_block - x_true) / np.linalg.norm(x_true)
        
        std_times.append(std_time)
        block_times.append(block_time)
        std_errors.append(std_error)
        block_errors.append(block_error)
    
    # Store averaged results
    scalability_results.append({
        'size': size,
        'avg_std_time': np.mean(std_times),
        'avg_block_time': np.mean(block_times),
        'avg_std_error': np.mean(std_errors),
        'avg_block_error': np.mean(block_errors)
    })

# Plot scalability results
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Time comparison
ax1.plot([r['size'] for r in scalability_results], 
         [r['avg_std_time'] for r in scalability_results], 
         'o-', label='Standard HP-INV', linewidth=2, markersize=6)
ax1.plot([r['size'] for r in scalability_results], 
         [r['avg_block_time'] for r in scalability_results], 
         's-', label='Block HP-INV', linewidth=2, markersize=6)
ax1.set_xlabel('Matrix Size')
ax1.set_ylabel('Average Time (s)')
ax1.set_title('Computation Time vs Matrix Size')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Error comparison
ax2.plot([r['size'] for r in scalability_results], 
         [r['avg_std_error'] for r in scalability_results], 
         'o-', label='Standard HP-INV', linewidth=2, markersize=6)
ax2.plot([r['size'] for r in scalability_results], 
         [r['avg_block_error'] for r in scalability_results], 
         's-', label='Block HP-INV', linewidth=2, markersize=6)
ax2.set_xlabel('Matrix Size')
ax2.set_ylabel('Average Relative Error')
ax2.set_yscale('log')
ax2.set_title('Solution Accuracy vs Matrix Size')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Time efficiency improvement
time_ratios = [r['avg_std_time'] / max(r['avg_block_time'], 1e-8) for r in scalability_results]
ax3.bar(range(len(matrix_sizes)), time_ratios, tick_label=matrix_sizes, alpha=0.7)
ax3.set_xlabel('Matrix Size')
ax3.set_ylabel('Time Ratio (Std / Block)')
ax3.set_title('Time Efficiency Improvement with BlockAMC')
ax3.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, v in enumerate(time_ratios):
    ax3.text(i, v + 0.01, f'{v:.2f}x', ha='center', va='bottom', fontsize=9)

# Error ratio
error_ratios = [max(r['avg_block_error'], 1e-8) / max(r['avg_std_error'], 1e-8) for r in scalability_results]
ax4.bar(range(len(matrix_sizes)), error_ratios, tick_label=matrix_sizes, alpha=0.7, color='orange')
ax4.set_xlabel('Matrix Size')
ax4.set_ylabel('Error Ratio (Block / Std)')
ax4.set_title('Accuracy Comparison (Lower is Better)')
ax4.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, v in enumerate(error_ratios):
    ax4.text(i, v + 0.01 if v > 0 else 0.01, f'{v:.2f}x', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

# Summary
print("\nScalability Summary:")
for r in scalability_results:
    time_efficiency = r['avg_std_time'] / max(r['avg_block_time'], 1e-8)
    print(f"{r['size']}x{r['size']}: Block is {time_efficiency:.2f}x faster, "
          f"error is {max(r['avg_block_error'], 1e-8)/max(r['avg_std_error'], 1e-8):.2f}x")

## 5. Memory Usage Analysis

Let's also compare memory usage between standard and block approaches.

In [ ]:
# Create matrices of different sizes and estimate memory usage
import sys

def estimate_memory_usage(matrix_size, block_size=None):
    """Estimate memory usage for matrix operations"""
    n = matrix_size
    
    # For standard approach: need to store the full matrix
    full_matrix_mem = n * n * 8  # 8 bytes per float64
    
    if block_size is None or block_size >= n:
        # Standard approach
        return full_matrix_mem
    else:
        # Block approach: only need to store blocks at a time
        block_matrix_mem = block_size * block_size * 8
        # Also need some overhead for the full matrix
        return block_matrix_mem + full_matrix_mem  # Simplified estimate

# Calculate memory usage for different approaches
matrix_sizes = [8, 16, 24, 32, 40, 48, 56, 64]
block_size = 8

std_memory = [estimate_memory_usage(size) for size in matrix_sizes]
block_memory = [estimate_memory_usage(size, block_size) for size in matrix_sizes]

# Calculate ratios
memory_ratios = [s/b for s, b in zip(std_memory, block_memory)]

# Create memory analysis plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Memory usage comparison
ax1.plot(matrix_sizes, std_memory, 'o-', label='Standard Approach', linewidth=2, markersize=6)
ax1.plot(matrix_sizes, block_memory, 's-', label=f'Block Approach (block size={block_size})', linewidth=2, markersize=6)
ax1.set_xlabel('Matrix Size')
ax1.set_ylabel('Memory Usage (bytes)')
ax1.set_title('Memory Usage Comparison')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Memory efficiency ratio
ax2.plot(matrix_sizes, memory_ratios, '^-', linewidth=2, markersize=6)
ax2.set_xlabel('Matrix Size')
ax2.set_ylabel('Memory Efficiency Ratio')
ax2.set_title('Memory Efficiency (Standard / Block)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nMemory Usage Summary:")
for i, size in enumerate(matrix_sizes):
    std_bytes = std_memory[i]
    block_bytes = block_memory[i]
    ratio = std_bytes / block_bytes
    print(f"{size}x{size}: Standard={std_bytes/1024:.1f}KB, Block={block_bytes/1024:.1f}KB, "
          f"{ratio:.2f}x more efficient")

## Summary

This notebook demonstrated the BlockAMC approach to scalability in the Analogico project:
1. The block-based approach can handle matrices larger than the physical RRAM array size
2. Performance analysis shows different trade-offs with various block sizes
3. Scalability testing reveals how the approach performs with increasing matrix sizes
4. Memory efficiency is important for large-scale implementations

The BlockAMC algorithm enables large-scale analogue matrix computations by partitioning problems into smaller subproblems that fit within the constraints of physical RRAM arrays.